<a href="https://colab.research.google.com/github/kwafafoa/neqsimpython/blob/master/DewpointCalc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install neqsim


     |████████████████████████████████| 30.7 MB 126 kB/s 
     |████████████████████████████████| 448 kB 11.8 MB/s 


In [2]:
import neqsim
from neqsim.thermo.thermoTools import*
import pandas as pd


In [133]:
inputdata = {
    'feedFlowRateTrain1': 11411.9,
    'feedPressure': 89.0,
    'feedTemperature':5.0,
    'sep1pressure': 85.0,
    'cooler1T': -5.0,
    'glycolflow': 1.0,
    'expOutPressure':67.0
  
    }

In [111]:
feedFluid = {'ComponentName':  ['water','MEG','methane','ethane','C6','C7'],
             'MolarComposition[-]':[1.0, 0.0, 0.5, 0.1, 0.1, 0.3],
             'MolarMass[kg/mol]': [None, None, None, None, 0.091, 0.19],
             'RelativeDensity[-]': [None, None, None, None, 0.7, 0.86]
}
reservoirFluidf = pd.DataFrame(feedFluid)
print(reservoirFluidf.head(20).to_string())
fluid7 = fluid_df(reservoirFluidf)

glycolFluid = fluid7.clone()
glycolFluid.setMolarComposition([0.0, 1.0, 0.0, 0.0, 0.0, 0.0])

  ComponentName  MolarComposition[-]  MolarMass[kg/mol]  RelativeDensity[-]
0         water                  1.0                NaN                 NaN
1           MEG                  0.0                NaN                 NaN
2       methane                  0.5                NaN                 NaN
3        ethane                  0.1                NaN                 NaN
4            C6                  0.1              0.091                0.70
5            C7                  0.3              0.190                0.86


In [144]:
from neqsim.process import clearProcess, runProcess, separator3phase, stream, valve, cooler, expander, mixer
clearProcess()
feedStream = stream(fluid7)
feedStream.setFlowRate(inputdata['feedFlowRateTrain1'], 'kg/hr')
feedStream.setPressure(inputdata['feedPressure'], 'barg')
feedStream.setTemperature(inputdata['feedTemperature'],'C')

glycolfeedStream = stream(glycolFluid)



slugcatcher = separator3phase(feedStream)
gasFromslugcatcher = stream(slugcatcher.getGasOutStream())

valve1 = valve(gasFromslugcatcher)
valve1.setOutletPressure(inputdata['sep1pressure'], 'barg')

sep1 = separator3phase(valve1.getOutStream())

cooler1 = cooler(sep1.getGasOutStream())
cooler1.setOutTemperature(inputdata['cooler1T'], 'C')

sep2 = separator3phase(cooler1.getOutStream())

mixer1 = mixer()
mixer1.addStream(sep2.getGasOutStream())
mixer1.addStream(glycolfeedStream)

expander1 = expander(mixer1.getOutStream(),inputdata['expOutPressure'])

sep3 = separator3phase(expander1.getOutStream())

gasexport = stream(sep3.getGasOutStream())
























runProcess()

In [149]:
printFrame(sep3.getFluid())

gasexport.setPressure(70.0,'bara')
print('hydrate/water dew point temperature',(gasexport.getHydrateEquilibriumTemperature()-273.15))


                        total               gas               oil           aqueous                 
             water 4.68785E-5        4.42998E-8        5.52266E-7        2.59041E-4  [mole fraction]
               MEG 1.80643E-1        6.62357E-7        2.25046E-4        9.98966E-1  [mole fraction]
           methane 7.60531E-1        9.28266E-1        2.94011E-1        7.14021E-4  [mole fraction]
            ethane 5.81078E-2        7.09207E-2         1.1901E-1        6.11017E-5  [mole fraction]
             C6_PC 6.68475E-4        8.11682E-4        3.22828E-1        6.13866E-9  [mole fraction]
             C7_PC 3.78753E-6        1.05446E-6        2.63925E-1       2.69622E-16  [mole fraction]
                                                                                                    
           Density                    6.34633E1         7.28972E2         1.25201E3         [kg/m^3]
     PhaseFraction                    8.1916E-1         1.1078E-5        1.80829E-1  [mole 